In [ ]:
import os
import audeer
import audonnx
import audinterface

# Model Playground

In [ ]:
# Load the model
model_root = 'model'
cache_root = 'cache'

audeer.mkdir(cache_root)
def cache_path(file):
    return os.path.join(cache_root, file)

url = 'https://zenodo.org/record/6221127/files/w2v2-L-robust-12.6bc4a7fd-1.1.0.zip'
dst_path = cache_path('model.zip')

if not os.path.exists(dst_path):
    audeer.download_url(url, dst_path, verbose=True,)
if not os.path.exists(model_root):
    audeer.extract_archive(dst_path, model_root, verbose=True,)

In [ ]:
model = audonnx.load(model_root)
print(model)

In [ ]:
sampling_rate = 16000
# use AudInterface to create interface to predict valence, dominance and arousal (logits layer)
interface = audinterface.Feature(
    model.labels('logits'),
    process_func=model,
    process_func_args={
        'outputs': 'logits',
    },
    sampling_rate=sampling_rate, # automatic resampling if sampling rates don't align
    resample=True,
    verbose=True,
)

In [ ]:
sample = "/data/dst_tsst_22_bi_multi_nt_lab/processed/audio_files/JK261022_tsst_video_segment.mp3"
interface.process_files(sample)